## GrEx 3-Hotel Customers Have Their Say About Brands ##

## Part 1 ##

<h3> Steps </h3>
> 1. We are provided a zip file called 'hotelCustsSp2018.zip' which has 26 JSON files in it
> 2. Each JSON file contains TripAdvisor reviews and other information about one hotel 
> 3. Prior to using Python, we need to unzip the JSON files to a 'hotelCustsSp2018' directory on our local machine
> 4. Next, we will write a loop that creates a DataFrame for each JSON file, which will result in a list of DataFrames
> 5. Finally, we will concatenate all those DataFrames to make one summary DataFrame, clean some of the data and calculate summary statistics 

In [183]:
# We need to import all our packages
import pandas as pd
import os
from pandas.io.json import json_normalize
import re
import json
from bs4 import BeautifulSoup
import numpy as np
from nltk import *
from nltk.corpus import stopwords
from nltk import word_tokenize
import string
from collections import Counter
pd.options.mode.chained_assignment = None  # default='warn'

In [184]:
# After unzipping the JSON files to our local machine, let's take a look at our hotelCustsSp2018 directory to ensure our JSON files are there 
# We can see that 26 files exist--one for each hotel
# Also, keep in mind that each file size is different which indicates there may be some irregularity with the data within the JSON files 
!dir hotelCustsSp2018 | findstr json

05/13/2018  05:55 PM            52,320 100506.json
05/13/2018  05:55 PM             1,184 1217974.json
05/13/2018  05:55 PM         1,407,821 150849.json
05/13/2018  05:55 PM           183,832 214680.json
05/13/2018  05:55 PM            59,905 240124.json
05/13/2018  05:55 PM           113,653 2515575.json
05/13/2018  05:55 PM           219,634 287670.json
05/13/2018  05:55 PM           208,767 550994.json
05/13/2018  05:55 PM               486 655424.json
05/13/2018  05:55 PM             1,005 677703.json
05/13/2018  05:55 PM           246,390 72572.json
05/13/2018  05:55 PM           123,437 72579.json
05/13/2018  05:55 PM           167,695 72586.json
05/13/2018  05:55 PM            42,330 72598.json
05/13/2018  05:55 PM            86,963 73393.json
05/13/2018  05:55 PM            46,804 73644.json
05/13/2018  05:55 PM            38,825 73706.json
05/13/2018  05:55 PM            25,246 73712.json
05/13/2018  05:55 PM            61,505 73718.json
05/13/2018  05:55 PM            50,488

In [185]:
# We will save our directory path to a variable called path which will later be used in our loop
path = ".\\hotelCustsSp2018\\"

In [186]:
# Using os and our path variable, we will create a list that has the file names of all the JSON files in our directory 
file_list = os.listdir(path)
file_list

['100506.json',
 '1217974.json',
 '150849.json',
 '214680.json',
 '240124.json',
 '2515575.json',
 '287670.json',
 '550994.json',
 '655424.json',
 '677703.json',
 '72572.json',
 '72579.json',
 '72586.json',
 '72598.json',
 '73393.json',
 '73644.json',
 '73706.json',
 '73712.json',
 '73718.json',
 '73727.json',
 '73739.json',
 '73743.json',
 '73751.json',
 '73757.json',
 '73760.json',
 '73768.json']

In [187]:
# Before we make our loop, let's explore one of our JSON files -- '72572.json'
# We will use 'open' to read the JSON file and then 'load' it into a dictionary variable called jsondat
with open('.\\hotelCustsSp2018\\72572.json') as input_file:
    jsondat=json.load(input_file)

In [188]:
# Let's see the names of our dictionary keys
jsondat.keys()

dict_keys(['Reviews', 'HotelInfo'])

In [189]:
# We will explore the values of the 'HotelInfo' key 
# The values are keys since 'HotelInfo' is also a dictionary
hotel_info = jsondat['HotelInfo']
hotel_info.keys()

dict_keys(['Name', 'HotelURL', 'Price', 'Address', 'HotelID', 'ImgURL'])

In [190]:
# Now we will create our loop
# The goal is to create hotel_ratings_list which is a list of 26 DataFrames (one for each hotel)
# The first part of the loop will create the jsondat dictionary and then will create a DataFrame from jsondat using the 'json_normalize' function 
# We will use 'if' statements to add the columns we want to each DataFrame depending if the values exist in jsondat
# Lastly, we will rename some columns and then add each DataFrame to a list which will result in a list of 26 DataFrames
hotel_ratings_list = []
for file in file_list:
    with open(path+file) as input_file:
        jsondat = json.load(input_file)
        hotel_df = json_normalize(jsondat['Reviews'])
        if 'HotelID' in jsondat['HotelInfo']:
            hotel_df.insert(0, 'Hotel ID', jsondat['HotelInfo']['HotelID'])
        if 'Name' in jsondat['HotelInfo']:
            hotel_df.insert(1, 'Hotel Name', jsondat['HotelInfo']['Name'])
        if 'Address' in jsondat['HotelInfo']:
            hotel_df.insert(2, 'Hotel Address', jsondat['HotelInfo']['Address'])
        hotel_df.rename(columns={'Ratings.Business service (e.g., internet access)':'Ratings.Business service'}, inplace=True)
        hotel_df.rename(columns={'ReviewID':'Review ID'}, inplace=True)
        hotel_df.rename(columns=lambda x: re.sub(r'^Ratings.','',x),inplace=True)
        hotel_ratings_list.append(hotel_df)

In [191]:
# Check to see how many DataFrames are in hotel_ratings_list
# There should be 26 -- one for each hotel 
len(hotel_ratings_list)

26

In [192]:
# We will now combine all 26 DataFrames from the hotel_ratings_list into one DataFrame
# This DataFrame will stay with these columns in case we need to use some of the data to create a separate DataFrame later 
pre_hotel_ratings_df = pd.concat(hotel_ratings_list)

In [193]:
# Show all the columns in pre_hotel_ratings_df and the first 5 rows of data
pre_hotel_ratings_df.head(5)

,Author,AuthorLocation,Business service,Check in / front desk,Cleanliness,Content,Date,Hotel Address,Hotel ID,Hotel Name,Location,Overall,Review ID,Rooms,Service,Sleep Quality,Title,Value
0,luvsroadtrips,"Arlington, WA",NaN,NaN,1,This place is not even suitable for the homele...,"January 3, 2012","<address class=""addressReset""> <span rel=""v:ad...",100506,Hotel Seattle,5,1.0,UR122476164,1,1,1,“You've got to be kidding me??!”,1
1,estelle e,"Vancouver, Canada",NaN,NaN,4,We stayed in downtown hotel Seattle for two ni...,"December 29, 2011","<address class=""addressReset""> <span rel=""v:ad...",100506,Hotel Seattle,5,4.0,UR122239883,3,4,5,"“great service, comfortable rooms, easy stay.”",3
2,RobertEddy,"San Diego, California",NaN,NaN,2,"i made reservations and when i showed up, i qu...","December 20, 2011","<address class=""addressReset""> <span rel=""v:ad...",100506,Hotel Seattle,1,1.0,UR121931325,1,1,1,“WTF!!!!!”,1
3,James R,"Chicago, Illinois",NaN,NaN,1,This hotel is so bad it's a joke. I could bare...,"October 30, 2011","<address class=""addressReset""> <span rel=""v:ad...",100506,Hotel Seattle,1,1.0,UR119896310,1,1,1,“Do Not Stay Here”,1
4,Shobha49,"Mumbai (Bombay), India",NaN,NaN,NaN,My husband and I stayed at this hotel from 16t...,"September 14, 2011","<address class=""addressReset""> <span rel=""v:ad...",100506,Hotel Seattle,5,1.0,UR118110693,1,1,NaN,“bad experience”,3


In [194]:
# We will select some of the columns from pre_hotel_ratings_df to create a new DataFrame for further analysis 
rating_select = ['Hotel ID', 'Hotel Name', 'Hotel Address', 'Review ID', 'Business service', 'Check in / front desk', 'Cleanliness', 'Location', 'Overall', 'Rooms', 'Service', 'Sleep Quality', 'Value']
hotel_ratings_df = pre_hotel_ratings_df[rating_select]
hotel_ratings_df.head(5)

,Hotel ID,Hotel Name,Hotel Address,Review ID,Business service,Check in / front desk,Cleanliness,Location,Overall,Rooms,Service,Sleep Quality,Value
0,100506,Hotel Seattle,"<address class=""addressReset""> <span rel=""v:ad...",UR122476164,NaN,NaN,1,5,1.0,1,1,1,1
1,100506,Hotel Seattle,"<address class=""addressReset""> <span rel=""v:ad...",UR122239883,NaN,NaN,4,5,4.0,3,4,5,3
2,100506,Hotel Seattle,"<address class=""addressReset""> <span rel=""v:ad...",UR121931325,NaN,NaN,2,1,1.0,1,1,1,1
3,100506,Hotel Seattle,"<address class=""addressReset""> <span rel=""v:ad...",UR119896310,NaN,NaN,1,1,1.0,1,1,1,1
4,100506,Hotel Seattle,"<address class=""addressReset""> <span rel=""v:ad...",UR118110693,NaN,NaN,NaN,5,1.0,1,1,NaN,3


In [195]:
# Next we wil remove the html tags from the Hotel Address column using Beautiful Soup
# This code will create beautify function
def beautify(s): 
    if isinstance (s, str):
        soup=BeautifulSoup(s, 'html.parser') 
        clean_string = soup.get_text().strip() 
        return clean_string
    else:
        return s

In [196]:
# Apply the beautify function and check to see if html tags were removed
hotel_ratings_df['Hotel Address']=hotel_ratings_df['Hotel Address'].apply(beautify)
hotel_ratings_df.head(5)

,Hotel ID,Hotel Name,Hotel Address,Review ID,Business service,Check in / front desk,Cleanliness,Location,Overall,Rooms,Service,Sleep Quality,Value
0,100506,Hotel Seattle,"315 Seneca St., Seattle, WA 98101",UR122476164,NaN,NaN,1,5,1.0,1,1,1,1
1,100506,Hotel Seattle,"315 Seneca St., Seattle, WA 98101",UR122239883,NaN,NaN,4,5,4.0,3,4,5,3
2,100506,Hotel Seattle,"315 Seneca St., Seattle, WA 98101",UR121931325,NaN,NaN,2,1,1.0,1,1,1,1
3,100506,Hotel Seattle,"315 Seneca St., Seattle, WA 98101",UR119896310,NaN,NaN,1,1,1.0,1,1,1,1
4,100506,Hotel Seattle,"315 Seneca St., Seattle, WA 98101",UR118110693,NaN,NaN,NaN,5,1.0,1,1,NaN,3


In [197]:
# We need to calculate some statistics for each of the ratings categories
# However, we need to first check for incorrect values in the ratings categories
# We need to deal with the -1 values since they will skew our summary statistics
hotel_ratings_df[['Business service','Check in / front desk', 'Cleanliness', 'Location', 'Overall', 'Rooms', 'Service', 'Sleep Quality', 'Value']].apply(pd.value_counts, dropna=False)

,Business service,Check in / front desk,Cleanliness,Location,Overall,Rooms,Service,Sleep Quality,Value
NaN,1407.0,1321.0,145.0,339.0,NaN,269.0,145.0,1680.0,132.0
-1,595.0,429.0,242.0,431.0,NaN,235.0,243.0,NaN,253.0
5,139.0,331.0,951.0,953.0,328.0,554.0,932.0,315.0,860.0
3,127.0,128.0,282.0,206.0,107.0,418.0,291.0,128.0,288.0
4,120.0,184.0,587.0,430.0,300.0,618.0,564.0,233.0,603.0
2,54.0,55.0,120.0,74.0,93.0,209.0,136.0,52.0,170.0
1,43.0,37.0,158.0,52.0,59.0,182.0,174.0,77.0,179.0
5.0,NaN,NaN,NaN,NaN,542.0,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,505.0,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,217.0,NaN,NaN,NaN,NaN


In [198]:
# We also need to change the data types of the rating categories to float types so statistics can be calculated
# We will do this with the help of the astype method
hotel_ratings_df[['Business service','Check in / front desk', 'Cleanliness', 'Location', 'Overall', 'Rooms', 'Service', 'Sleep Quality', 'Value']]=hotel_ratings_df[['Business service','Check in / front desk', 'Cleanliness', 'Location', 'Overall', 'Rooms', 'Service', 'Sleep Quality', 'Value']].astype(float)

In [199]:
# Let's check to see if the data types for the rating categories have been updated to float type
hotel_ratings_df.dtypes

Hotel ID                  object
Hotel Name                object
Hotel Address             object
Review ID                 object
Business service         float64
Check in / front desk    float64
Cleanliness              float64
Location                 float64
Overall                  float64
Rooms                    float64
Service                  float64
Sleep Quality            float64
Value                    float64
dtype: object

In [200]:
# Let's now replace all -1 values with NaN using replace method
hotel_ratings_df['Business service'].replace(-1, np.nan, inplace=True)
hotel_ratings_df['Check in / front desk'].replace(-1, np.nan, inplace=True)
hotel_ratings_df['Cleanliness'].replace(-1, np.nan, inplace=True)
hotel_ratings_df['Location'].replace(-1, np.nan, inplace=True)
hotel_ratings_df['Overall'].replace(-1, np.nan, inplace=True)
hotel_ratings_df['Rooms'].replace(-1, np.nan, inplace=True)
hotel_ratings_df['Service'].replace(-1, np.nan, inplace=True)
hotel_ratings_df['Sleep Quality'].replace(-1, np.nan, inplace=True)
hotel_ratings_df['Value'].replace(-1, np.nan, inplace=True)

In [201]:
# Now let's check our ratings categories to see if the -1 values are replaced 
hotel_ratings_df[['Business service','Check in / front desk', 'Cleanliness', 'Location', 'Overall', 'Rooms', 'Service', 'Sleep Quality', 'Value']].apply(pd.value_counts, dropna=False)

,Business service,Check in / front desk,Cleanliness,Location,Overall,Rooms,Service,Sleep Quality,Value
1.0,43,37,158,52,249.0,182,174,77,179
2.0,54,55,120,74,237.0,209,136,52,170
3.0,127,128,282,206,324.0,418,291,128,288
4.0,120,184,587,430,805.0,618,564,233,603
5.0,139,331,951,953,870.0,554,932,315,860
NaN,2002,1750,387,770,NaN,504,388,1680,385


In [202]:
# Now we can compute summary statistics using the 'describe' method on hotel_ratings_df
# The describe method will only display statistics for the rating categories
hotel_ratings_df.describe(include=[np.number])

,Business service,Check in / front desk,Cleanliness,Location,Overall,Rooms,Service,Sleep Quality,Value
count,483.000000,735.000000,2098.000000,1715.000000,2485.000000,1981.000000,2097.000000,805.000000,2100.000000
mean,3.534161,3.975510,3.978551,4.258309,3.728370,3.582029,3.927039,3.816149,3.854762
std,1.259146,1.173945,1.222511,1.027217,1.300262,1.251246,1.258907,1.279765,1.271729
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,3.000000,3.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000
50%,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,4.000000,4.000000
75%,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [203]:
# As one last item, we will display how many reviews were left per hotel 
hotel_ratings_df[['Hotel ID']].apply(pd.value_counts, dropna=False)

,Hotel ID
150849,710
72572,233
287670,188
214680,174
550994,167
72586,137
72579,113
2515575,96
73393,93
73739,60


In [204]:
# Display first five rows of our final DataFrame - hotel_ratings_df
hotel_ratings_df.head(5)

,Hotel ID,Hotel Name,Hotel Address,Review ID,Business service,Check in / front desk,Cleanliness,Location,Overall,Rooms,Service,Sleep Quality,Value
0,100506,Hotel Seattle,"315 Seneca St., Seattle, WA 98101",UR122476164,NaN,NaN,1.0,5.0,1.0,1.0,1.0,1.0,1.0
1,100506,Hotel Seattle,"315 Seneca St., Seattle, WA 98101",UR122239883,NaN,NaN,4.0,5.0,4.0,3.0,4.0,5.0,3.0
2,100506,Hotel Seattle,"315 Seneca St., Seattle, WA 98101",UR121931325,NaN,NaN,2.0,1.0,1.0,1.0,1.0,1.0,1.0
3,100506,Hotel Seattle,"315 Seneca St., Seattle, WA 98101",UR119896310,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,100506,Hotel Seattle,"315 Seneca St., Seattle, WA 98101",UR118110693,NaN,NaN,NaN,5.0,1.0,1.0,1.0,NaN,3.0


## Part 2 ##

<h3> Steps </h3>
> 1. The main goal of Part 2 is to create the hotWords dictionary 
> 2. A loop will utilized to create a frequency dictionary of "content" words for each hotel 
> 3. The "content" words will come from the Content category of a DataFrame we create within the loop 
> 4. We will add our frequency dictionary of "content" words to each hotel's respective Hotel ID and then use the Hotel ID as keys in our hotWords dictionary 
> 5. At the end, we will save the hotWords dictionary into a JSON file and report the number of unique content words in each of the hotel's dicts

In [205]:
# We first will define 'get_dictionary' function which will be used within our loop to create each dictionary that will go in hotWords
# Now let's begin creating our hotWords dictionary by setting it up as an empty dictionary 
# Similar to Part 1, we created a hotel_df in the beginning part of our loop
# Our 'reviews' variable is created by concatenating all the strings in the Content column of hotel_df
# Next, we obtain our dictionary of 'content' words using 'get_dictionary'
# The last step is to create a key/value pair using the value from 'HotelID' as the key and 'word_dict' as the value
# The loop iterates 26 times to create the hotWords dictionary which will have 26 dictionaries -- one for each hotel 

def get_dictionary(text):
    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # Remove non-alphabetic tokens
    words = [word for word in stripped if word.isalpha()]
    # Remove stop words from our dictionary which is one of its main strengths 
    # A weakness of our dictionary is that we didn't do any stemming so it will include 'walking' and 'walk'
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if not word in stop_words]
    return dict(Counter(words).most_common())

hotWords = {}
# We also want to create 'uniqueWords' dictionary that shows how many unique words are in each dictionary
uniqueWords = {}
for file in file_list:      
    with open(path+file) as input_file:
        jsondat = json.load(input_file)
        hotel_df = json_normalize(jsondat['Reviews']) 
        reviews = hotel_df.Content.str.cat(sep=",")         
        word_dict = get_dictionary(reviews)
        unique_count = len(word_dict.keys())
        hotel_id = jsondat['HotelInfo']['HotelID']
        hotWords[hotel_id] = word_dict
        uniqueWords[hotel_id] = unique_count

In [206]:
# Check the length of hotWords dictionary to see that it has 26 keys--one for each hotel
len(hotWords)

26

In [207]:
# Print out the keys of hotWords dictionary to confirm the keys are the Hotel Ids for the 26 hotels
hotWords_keys = hotWords.keys()
list(hotWords_keys)

['100506',
 '1217974',
 '150849',
 '214680',
 '240124',
 '2515575',
 '287670',
 '550994',
 '655424',
 '677703',
 '72572',
 '72579',
 '72586',
 '72598',
 '73393',
 '73644',
 '73706',
 '73712',
 '73718',
 '73727',
 '73739',
 '73743',
 '73751',
 '73757',
 '73760',
 '73768']

In [208]:
# Let's inspect the values ('Hot Words') for Hotel ID 72572 to see an application of hotWords dictionary 
hotWords_72572 = hotWords['72572']
hotWords_72572

{'hotel': 503,
 'room': 254,
 'seattle': 202,
 'staff': 172,
 'great': 166,
 'breakfast': 160,
 'pioneer': 149,
 'square': 149,
 'stay': 139,
 'location': 109,
 'good': 105,
 'clean': 104,
 'best': 98,
 'would': 94,
 'friendly': 93,
 'nt': 91,
 'helpful': 90,
 'free': 90,
 'walk': 89,
 'us': 89,
 'one': 87,
 'nice': 86,
 'rooms': 83,
 'stayed': 83,
 'night': 82,
 'area': 81,
 'western': 77,
 'also': 76,
 'walking': 76,
 'place': 75,
 'market': 68,
 'close': 68,
 'comfortable': 67,
 'parking': 66,
 'street': 64,
 'downtown': 64,
 'blocks': 61,
 'front': 60,
 'within': 59,
 'old': 58,
 'well': 58,
 'get': 56,
 'day': 55,
 'city': 55,
 'service': 54,
 'waterfront': 53,
 'bus': 53,
 'back': 53,
 'pike': 52,
 'small': 52,
 'away': 52,
 'block': 49,
 'distance': 49,
 'restaurants': 48,
 'desk': 48,
 'located': 47,
 'large': 47,
 'nights': 45,
 'could': 44,
 'right': 44,
 'around': 44,
 'price': 43,
 'easy': 42,
 'found': 42,
 'historic': 41,
 'next': 41,
 'bed': 40,
 'little': 40,
 'quiet': 

In [209]:
# We will now save the hotWords dictionary as a JSON file
with open('hotWords.json', 'w') as output_file:
    json.dump(hotWords, output_file)

In [210]:
# Let's read our JSON file back in and check to see that it is correct by using a Boolean comparison 
# Note that the items in a dictionary are not supposed to have any order, so using a Boolean comparison simply checks if they have 
# the same content 
# Output should be True if both dictionaries are the same 
with open('hotWords.json') as input_file:
    verify_hotWords = json.load(input_file)
verify_hotWords==hotWords

True

In [211]:
# One final task that will completed is reporting the number of unique content words from each hotel's dictionary
# Note that we already calculated this as uniqueWords in our loop so we will just display it here
# For example, we can see that Hotel ID 72572 has 3046 unique content words
uniqueWords

{'100506': 1395,
 '1217974': 56,
 '150849': 11304,
 '214680': 2775,
 '240124': 1414,
 '2515575': 2122,
 '287670': 3018,
 '550994': 3007,
 '655424': 5,
 '677703': 44,
 '72572': 3046,
 '72579': 2160,
 '72586': 2715,
 '72598': 1137,
 '73393': 1700,
 '73644': 1184,
 '73706': 927,
 '73712': 806,
 '73718': 1500,
 '73727': 1336,
 '73739': 1571,
 '73743': 1016,
 '73751': 654,
 '73757': 880,
 '73760': 1112,
 '73768': 772}